# Step-1: Loading the Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import glob 
filePaths = glob.glob("/home/prakhar/Desktop/College/3rdYear/SEM-VI/MinorProject/Datasets/archive-2/*.csv")
df_list = [pd.read_csv(file,low_memory=False) for file in filePaths]
df=pd.concat(df_list,ignore_index=True)
print(df.shape)
print(df.head())

(2830743, 79)
    Destination Port   Flow Duration   Total Fwd Packets  \
0              54865               3                   2   
1              55054             109                   1   
2              55055              52                   1   
3              46236              34                   1   
4              54863               3                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                        0                           12   
1                        1                            6   
2                        1                            6   
3                        1                            6   
4                        0                           12   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                             0                       6   
1                             6                       6   
2                             6                       6   
3                             6   

In [3]:
df.columns = df.columns.str.strip()
df['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

# Step-2: Pre-Processing

In [4]:
df.dtypes #Viewing Datatype

Destination Port                 int64
Flow Duration                    int64
Total Fwd Packets                int64
Total Backward Packets           int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
Idle Std                       float64
Idle Max                         int64
Idle Min                         int64
Label                           object
Length: 79, dtype: object

In [5]:
is_inf = df.isin([np.inf, -np.inf])

inf_counts = is_inf.sum()

# Filter to show only columns with infinite values
columns_with_inf = inf_counts[inf_counts > 0]
print("Columns with infinite values:\n", columns_with_inf)

nan_counts = df.isnull().sum()

# Filter to show only columns with NaN values
columns_with_nan = nan_counts[nan_counts > 0]
print("\nColumns with NaN values:\n", columns_with_nan)

combined_counts = nan_counts + inf_counts

# Filter to show only columns with any NaN or inf values
columns_with_nan_or_inf = combined_counts[combined_counts > 0]
print("\nColumns with NaN or infinite values:\n", columns_with_nan_or_inf)

duplicate_rows_count = df.duplicated().sum()
print("\nNumber of duplicate rows:", duplicate_rows_count)

Columns with infinite values:
 Flow Bytes/s      1509
Flow Packets/s    2867
dtype: int64

Columns with NaN values:
 Flow Bytes/s    1358
dtype: int64

Columns with NaN or infinite values:
 Flow Bytes/s      2867
Flow Packets/s    2867
dtype: int64

Number of duplicate rows: 308381


In [6]:
df.replace([np.inf, -np.inf], np.nan, inplace=True) #Replacing inf values
df.dropna(inplace=True) #Replacing NaN values
df.drop_duplicates(inplace=True) #Removing duplicates to improve accuracy
category_mapping = {
    'BENIGN': 'BENIGN',
    'Bot': 'BOTNET',
    'DDoS': 'DOS',
    'DoS GoldenEye': 'DOS',
    'DoS Hulk': 'DOS',
    'DoS Slowhttptest': 'DOS',
    'DoS slowloris': 'DOS',
    'FTP-Patator': 'BRUTE_FORCE',
    'SSH-Patator': 'BRUTE_FORCE',
    'Heartbleed': 'WEB_ATTACK',
    'Infiltration': 'WEB_ATTACK',
    'PortScan': 'RECONNAISSANCE',
    'Web Attack � Brute Force': 'WEB_ATTACK',
    'Web Attack � Sql Injection': 'WEB_ATTACK',
    'Web Attack � XSS': 'WEB_ATTACK'
}
#Applying Mapping to simplify labels
df['Label'] =df['Label'].map(category_mapping)

In [7]:
print(df['Label'].value_counts())  # Before encoding

Label
BENIGN            2095057
DOS                321759
RECONNAISSANCE      90694
BRUTE_FORCE          9150
WEB_ATTACK           2190
BOTNET               1948
Name: count, dtype: int64


In [8]:
from sklearn.preprocessing import LabelEncoder #After Encoding
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

In [9]:
print(df['Label'].value_counts())  # Before encoding

Label
0    2095057
3     321759
4      90694
2       9150
5       2190
1       1948
Name: count, dtype: int64


In [10]:
import pandas as pd

# Get class counts
class_counts = df['Label'].value_counts()

# Select classes with more than 5000 samples
selected_classes = class_counts[class_counts > 50000].index.tolist()

# Manually include "WEB_ATTACK" and "BOTNET" if they exist in the dataset
for attack in [1, 5, 2]:
    if attack in df['Label'].values and attack not in selected_classes:
        selected_classes.append(attack)

# Select rows where 'Label' is in selected_classes
selected = df[df['Label'].isin(selected_classes)]

dfs = []
for name in selected_classes:
    df_class = selected[selected['Label'] == name]

    # Downsample if the class has more than 5000 samples
    if len(df_class) > 50000:
        df_class = df_class.sample(n=50000, random_state=0)

    dfs.append(df_class)

# Combine all sampled data
balanced_df = pd.concat(dfs, ignore_index=True)

# Check final class distribution
print(balanced_df['Label'].value_counts())

Label
0    50000
3    50000
4    50000
2     9150
5     2190
1     1948
Name: count, dtype: int64


In [11]:
balanced_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,62874,4,2,116,186,29,29,29.000,0.000000,...,20,0.00000,0.00000,0,0,0.000,0.00000,0,0,0
1,57225,86,1,1,0,0,0,0,0.000,0.000000,...,32,0.00000,0.00000,0,0,0.000,0.00000,0,0,0
2,80,116003861,16,14,440,412,356,0,27.500,87.612784,...,20,46441.09091,31255.09835,140678,36852,9995557.727,14892.00092,10000000,9950679,0
3,443,5683083,8,6,397,3716,216,0,49.625,79.224703,...,20,269399.00000,0.00000,269399,269399,5413681.000,0.00000,5413681,5413681,0
4,53,23652,1,1,48,128,48,48,48.000,0.000000,...,20,0.00000,0.00000,0,0,0.000,0.00000,0,0,0


In [12]:
from imblearn.over_sampling import SMOTE

x = balanced_df.drop('Label', axis=1)
y = balanced_df['Label']

smote = SMOTE(sampling_strategy='auto', random_state=0)
X_upsampled, y_upsampled = smote.fit_resample(x, y)

blnc_data = pd.DataFrame(X_upsampled)
blnc_data['Label'] = y_upsampled
blnc_data = blnc_data.sample(frac=1)

print(blnc_data['Label'].value_counts())
blnc_data.head()

Label
3    50000
0    50000
2    50000
5    50000
1    50000
4    50000
Name: count, dtype: int64


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
96267,80,85813201,6,7,319,11595,319,0,53.166667,130.231205,...,32,4.0,0.0,4,4,85700000.0,0.0,85700000,85700000,3
22763,54817,3,2,0,12,0,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
221387,21,259,2,1,14,0,14,0,7.000000,9.899495,...,32,0.0,0.0,0,0,0.0,0.0,0,0,2
279392,80,5620616,3,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,5
172975,8080,996551,3,3,0,18,0,0,0.000000,0.000000,...,28,0.0,0.0,0,0,0.0,0.0,0,0,1


In [13]:
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
x = blnc_data.drop('Label',axis=1)
y = blnc_data['Label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)
n_components = 20
ipca = IncrementalPCA(n_components=n_components, batch_size=500)
x_pca = ipca.fit_transform(X_scaled) 
print(f"Explained Variance Ratio: {sum(ipca.explained_variance_ratio_):.4f}")
x = x_pca

Explained Variance Ratio: 0.9314


In [14]:
pca_df = pd.DataFrame(x, columns=[f'PC{i+1}' for i in range(x.shape[1])])
print(pca_df)

             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0       9.379800 -1.017785 -0.329970  0.997233 -1.339367 -1.051175  0.196905   
1      -2.038059 -0.205476 -0.869869  0.789847 -1.097982 -5.029906  0.807757   
2      -1.641709 -0.181755 -1.090432  0.475754 -0.635095 -3.132271  0.988217   
3      -1.529498 -0.121807 -0.474504 -0.134633  0.231976  1.935559  0.154752   
4      -1.805706 -0.052927 -0.083048 -0.040924  0.072483  0.646475 -0.219285   
...          ...       ...       ...       ...       ...       ...       ...   
299995 -1.984103 -0.024942  0.073328 -0.066094  0.097401  1.096866 -0.116713   
299996 -1.055640  0.350216  0.586640 -0.470618  0.687411  1.162520 -0.219952   
299997 -1.661999 -0.149434 -0.712333  0.328230 -0.431758 -1.063399  0.264804   
299998 -0.204051  0.139832  1.569397  0.263164 -0.428468  0.867197 -0.057815   
299999 -1.868631 -0.190460 -0.601571  0.558574 -0.775902 -3.356985  0.090546   

             PC8       PC9      PC10   

In [15]:
pca_df.shape

(300000, 20)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,shuffle=True)

# Step-3: Applying Classifier Algorithms

In [17]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import RocCurveDisplay

#### Step-3.1: Applying Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier #Importing Random Forest classifier
rf = RandomForestClassifier(n_estimators=1,max_depth=7,random_state=42)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test) #Performing Random Forest

In [ ]:
print(classification_report(y_test, y_pred)) #Printing Classification Report
class_indices = rf.classes_
print(class_indices)

In [ ]:
cv_rf = cross_val_score(rf, x_train, y_train, cv=5)
print(f'\nCross-validation scores:', ', '.join(map(str, cv_rf)))
print(f'\nMean cross-validation score: {cv_rf.mean():.2f}')

In [ ]:
print(rf.score(x_train,y_train)) #RF Score x_train,y_train

In [ ]:
print(rf.score(x_test, y_test)) #RF Score x_test,y_test

In [ ]:
# Compute ROC curve
# False Positive Rate, True Positive Rate
conf_mtr = confusion_matrix(y_true=y_test,y_pred=y_pred)
# Creating a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mtr, annot=True, cmap='Blues',
            xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()

In [ ]:
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i, class_name in enumerate(lb.classes_):
    y_score = rf.predict_proba(x_test)[:, i]
    RocCurveDisplay.from_predictions(
        y_test_bin[:, i],
        y_score,
        name=f'ROC curve for {class_name}',
        ax=plt.gca()
    )

plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

#### Step-3.2: Applying KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier #Importing KNN
knn = KNeighborsClassifier(n_neighbors= 100) #Changing Value Manually (Due to Performance Reasons)

knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)  # Performing Random Forest

In [ ]:
print(classification_report(y_test, y_pred)) #Printing Classification Report

In [ ]:
cv_knn = cross_val_score(knn, x_train, y_train, cv=5)
print(f'\nCross-validation scores:', ', '.join(map(str, cv_knn)))
print(f'\nMean cross-validation score: {cv_knn.mean():.2f}')

In [ ]:
print(knn.score(x_train, y_train)) #RF Score x_train,y_train

In [ ]:
print(knn.score(x_test, y_test)) #RF Score x_train,y_train

In [ ]:
# Compute Confusion Matrix
# False Positive Rate, True Positive Rate
conf_mtr = confusion_matrix(y_true=y_test, y_pred=y_pred)
# Creating a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mtr, annot=True, cmap='Blues',
            xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()

In [ ]:
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i, class_name in enumerate(lb.classes_):
    y_score = rf.predict_proba(x_test)[:, i]
    RocCurveDisplay.from_predictions(
        y_test_bin[:, i],
        y_score,
        name=f'ROC curve for {class_name}',
        ax=plt.gca()
    )

plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

#### Step-3.3: Applying DNN

In [18]:
import tensorflow as tf
import keras
from keras import layers

2025-03-17 23:42:37.405784: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 23:42:37.417585: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 23:42:37.425507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742235157.438163    9031 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742235157.441838    9031 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-17 23:42:37.455155: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [19]:
model = keras.Sequential()
model.add(layers.Input(shape=(x_train.shape[1],)))  # Input layer

model.add(layers.Dense(128,activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(256, activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(128, activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(6, activation='softmax'))  # Output layer

W0000 00:00:1742235160.599957    9031 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,382 (271.02 KB)

 Trainable params: 69,382 (271.02 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy']) #compiling model


In [22]:
x_val = x_train[:5000]
partial_x_train = x_train[5000:]

y_val = y_train[:5000]
partial_y_train = y_train[5000:]

model.fit(partial_x_train,partial_y_train,epochs=10,batch_size=16,validation_data=(x_val,y_val)) #Fitting Model
model.save("DCNN.keras") #Saving the model

Epoch 1/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.8839 - loss: 0.3972 - val_accuracy: 0.9608 - val_loss: 0.1245
Epoch 2/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9646 - loss: 0.1189 - val_accuracy: 0.9718 - val_loss: 0.1011
Epoch 3/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9711 - loss: 0.0976 - val_accuracy: 0.9742 - val_loss: 0.0901
Epoch 4/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.9731 - loss: 0.0890 - val_accuracy: 0.9774 - val_loss: 0.0814
Epoch 5/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9755 - loss: 0.0807 - val_accuracy: 0.9782 - val_loss: 0.0755
Epoch 6/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9769 - loss: 0.0763 - val_accuracy: 0.9774 - val_loss: 0.0736
Epoch 7/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9775 - loss: 0.0749 - val_accuracy: 0.9794 - val_loss: 0.0717
Epoch 8/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 

In [23]:
print(f"Results of Testing") #Testing the model
model.evaluate(x_test,y_test)

Results of Testing
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9806 - loss: 0.0576  


[0.05791729316115379, 0.9809333086013794]

In [26]:
print(f"Result of Prediction") #Performing Predictions
predictions=model.predict(x=x_test[:5])
print(predictions)
print("predictions shape:", predictions.shape)

Result of Prediction
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[6.0003311e-03 3.9960807e-10 5.6738139e-05 5.9939935e-05 1.4934037e-05
  9.9386805e-01]
 [4.8526068e-04 9.9949729e-01 1.7673479e-07 1.6624030e-06 1.5441197e-05
  1.6679508e-07]
 [9.9999988e-01 1.9214634e-12 2.5443024e-12 7.6540587e-08 8.0237674e-09
  1.7549844e-11]
 [5.9115705e-03 3.6490888e-10 5.4471573e-05 5.8319598e-05 1.3706192e-05
  9.9396187e-01]
 [6.0528005e-09 3.2992513e-14 1.0406902e-12 1.0000000e+00 3.1156003e-10
  8.1804494e-12]]
predictions shape: (5, 6)
